# MinHASH LSH

wyszukiwanie podobnych dokumentow i duplukatow uzywajac wyjątkowo małej reprezentacji dokumentów

In [2]:
# importujemy pyspark oraz tworzyny nowy kontekst (reprezentujący połączenie z serwerem spark). W tym przypadku
# serwer zostanie wystartowny automatycznie

import pyspark 
import pyspark.sql as psql

sc = pyspark.SparkContext()
spark_session = psql.SparkSession(sc)

Pierwszym krokiem jest tokenizacja [link](https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html]). Zrobimy to na zabawkowym przykładzie ale z wykorzystaniem rozproszenia oferowanego przez spark.

In [3]:
from pyspark.sql.functions import udf, col

# zbior dokumentow jako DataFrame a kazdy wiersz to bag-of-words

raw_docs = spark_session.createDataFrame([
    ("Ala ma kota a kot ma ale", ),
    ("Hania ma kangura ale nie ma kota", ),
    ("A to jest zupelnie inne zdanie", )
], ["text"])

raw_docs.show()

tokenizer = udf(lambda text: text.split()) #stworzmy tokenizer i zdefinujmy go jako user-defined-function

#nowa kolumna na podstawie 
tokenized_docs = raw_docs.withColumn('tokenized', tokenizer(col('text')))

tokenized_docs.show()
print (tokenized_docs.first())

+--------------------+
|                text|
+--------------------+
|Ala ma kota a kot...|
|Hania ma kangura ...|
|A to jest zupelni...|
+--------------------+

+--------------------+--------------------+
|                text|           tokenized|
+--------------------+--------------------+
|Ala ma kota a kot...|[Ala, ma, kota, a...|
|Hania ma kangura ...|[Hania, ma, kangu...|
|A to jest zupelni...|[A, to, jest, zup...|
+--------------------+--------------------+

Row(text='Ala ma kota a kot ma ale', tokenized='[Ala, ma, kota, a, kot, ma, ale]')


### Zadanie 1
wykorzystaj gotowy tokenizer z biblioteki ml.feature

In [4]:
import pyspark.ml.feature as feature

tokenizer = feature.Tokenizer(inputCol="text", outputCol="tokenized")

tokenized_docs = tokenizer.transform(raw_docs)

tokenized_docs.show()

# Przkład, na bazie którego się uczyłem (zakładka Python):
# https://spark.apache.org/docs/latest//ml-features.html#tokenizer

+--------------------+--------------------+
|                text|           tokenized|
+--------------------+--------------------+
|Ala ma kota a kot...|[ala, ma, kota, a...|
|Hania ma kangura ...|[hania, ma, kangu...|
|A to jest zupelni...|[a, to, jest, zup...|
+--------------------+--------------------+



Tak jak na zajęciach WDAM, pracowaliśmy z wektorową reprezentacją dokumentów [Vector-Space Model](https://en.wikipedia.org/wiki/Vector_space_model) tutaj zrobimy to samo przy uzyciu sparkowej biblioteki do uczenia maszynkowego (mllib), lecz zamiast znanego nam modelu TF-IDF uzyjemy skip-n-gram modelu [link](https://towardsdatascience.com/skip-gram-nlp-context-words-prediction-algorithm-5bbf34f84e0c) zaimplementowanego w obiekcie Word2Vec. Odsylam do dokumentacji [tutaj](https://spark.apache.org/docs/2.2.0/mllib-feature-extraction.html).

In [5]:
from pyspark.mllib.feature import Word2Vec

rdd = sc.parallelize( [
    "Ala ma kota a kot ma ale".lower().split(' '),
    "Hania ma kangura ale nie ma kota".lower().split(' '), 
    "A to jest zupelnie inne zdanie".lower().split(' ') ])
#mozemy uzyc pre-trenowanego modelu, ale w konteksice naszego problemu o wiele lepiej znalezc parametry modelu samemu
model = Word2Vec().setVectorSize(4).setMinCount(1)
model = model.fit(rdd)

print ('Zobaczmy jak wyglada wektorowa reprezentacja slow:')
print (model.getVectors())

#reprezentacja slowa jest zwrona jako gestwy wektor wielkosci 3
model.transform('ala')

Zobaczmy jak wyglada wektorowa reprezentacja slow:
{'nie': [0.114528336, -0.006846724, -0.025199931, 0.08501015], 'kangura': [0.063611336, 0.025590949, 0.029352859, 0.0075537614], 'inne': [-0.07924052, -0.121790625, -0.09923935, 0.12075767], 'jest': [-0.043344904, -0.046249412, -0.123785876, 0.09940506], 'a': [0.02048348, 0.03422871, -0.097975075, 0.092381984], 'zupelnie': [0.05157309, 0.06834473, 0.07105093, -0.040504053], 'ma': [0.11533597, -0.054423075, -0.04744574, 0.049611572], 'to': [-0.11152293, 0.084827356, -0.022310827, 0.07881487], 'kota': [-0.07808207, -0.0019579693, 0.123322286, 0.1223986], 'ale': [0.061333854, 0.047559403, 0.006485404, -0.053311795], 'kot': [-0.07221441, -0.016918432, 0.029325038, -0.109216824], 'ala': [-0.00823792, -0.054052386, 0.084926665, 0.028311351], 'hania': [-0.08199044, 0.017508304, 0.00076313416, 0.0066684536], 'zdanie': [0.04390236, 0.08001853, -0.036443405, -0.03994095]}


DenseVector([-0.0082, -0.0541, 0.0849, 0.0283])

W naszym zabawkowym przykladzie Vec2Model jest zbyt zlozonym modelem. Sprobjmy wykorzystac bardzo prosta reprezetancje wektorowa zliczajaca slowa w dokumencie 

In [6]:
from pyspark.ml.feature import CountVectorizer

cv_model = CountVectorizer(inputCol='tokenized', outputCol='features', 
                            minDF=1.0, vocabSize=100)
cv_model = cv_model.fit(tokenized_docs)
features_df = cv_model.transform(tokenized_docs)

features_df.show()

print (features_df.head())
print (cv_model.vocabulary)
print ('\nPierwszy dokument ma nastepujace zakodowane (w postaci rzadkiej reprezentacji) slowa')
print (features_df.first()['features'])
print ('to jest obiekt sparseVector, ktory reprezentuje wiersz rzadkiej macierzy o 14-tu kolumnach z wartosciami niezerowymi w [0,1,2,3,6,8]')

+--------------------+--------------------+--------------------+
|                text|           tokenized|            features|
+--------------------+--------------------+--------------------+
|Ala ma kota a kot...|[ala, ma, kota, a...|(14,[0,1,2,3,5,9]...|
|Hania ma kangura ...|[hania, ma, kangu...|(14,[0,1,3,4,7,11...|
|A to jest zupelni...|[a, to, jest, zup...|(14,[2,6,8,10,12,...|
+--------------------+--------------------+--------------------+

Row(text='Ala ma kota a kot ma ale', tokenized=['ala', 'ma', 'kota', 'a', 'kot', 'ma', 'ale'], features=SparseVector(14, {0: 2.0, 1: 1.0, 2: 1.0, 3: 1.0, 5: 1.0, 9: 1.0}))
['ma', 'ale', 'a', 'kota', 'nie', 'ala', 'zupelnie', 'kangura', 'inne', 'kot', 'jest', 'hania', 'to', 'zdanie']

Pierwszy dokument ma nastepujace zakodowane (w postaci rzadkiej reprezentacji) slowa
(14,[0,1,2,3,5,9],[2.0,1.0,1.0,1.0,1.0,1.0])
to jest obiekt sparseVector, ktory reprezentuje wiersz rzadkiej macierzy o 14-tu kolumnach z wartosciami niezerowymi w [0,1,2,3,6

In [7]:
from pyspark.ml.feature import MinHashLSH

mh_model = MinHashLSH(inputCol='features', outputCol='hashes', 
                      numHashTables=2)

#mh_model = MinHashLSH(inputCol='features', outputCol='hashes', numHashTables=6) #"startowo"

mh_model = mh_model.fit(features_df)

similarity = mh_model.approxSimilarityJoin(features_df, features_df, 1, distCol='JaccardDistance')

similarity.show()

+--------------------+--------------------+------------------+
|            datasetA|            datasetB|   JaccardDistance|
+--------------------+--------------------+------------------+
|[Ala ma kota a ko...|[Hania ma kangura...|0.6666666666666667|
|[Hania ma kangura...|[Ala ma kota a ko...|0.6666666666666667|
|[A to jest zupeln...|[A to jest zupeln...|               0.0|
|[Ala ma kota a ko...|[Ala ma kota a ko...|               0.0|
|[Hania ma kangura...|[Hania ma kangura...|               0.0|
+--------------------+--------------------+------------------+



### Zadanie 2
Jaka jest minimalna ilosc fukcji haszujacych aby wychwycic podobienstow 0.666(6) miedzy pierwszym a drugim dokumentem?

In [ ]:
Minimalna ilość fukcji haszujacych, aby
wychwycić podobieństwo 0.(6) między pierwszym,
a drugim dokumentem to: numHashTables=2.
    
Dla numHashTables=1 JaccardDistance daje
jeszcze wszędzie 0.0.

### Zadanie 3
wykorzystaj mh_model.approxNearestNeighbors aby znalezc 'podobny' dokument do new_doc


In [9]:
import pyspark.ml.feature as feature
from pyspark.ml.feature import CountVectorizer

new_doc = "Ala ma psa a nie ma kota"

new_doc_df = spark_session.createDataFrame([(new_doc, )], ["text"])

tokenizer_new = feature.Tokenizer(inputCol="text", outputCol="tokenized")
tokenized_docs_new = tokenizer_new.transform(new_doc_df)
tokenized_docs_new.show()

new_doc_features = cv_model.transform(tokenized_docs_new)

ret = mh_model.approxNearestNeighbors(features_df, new_doc_features.first()['features'],  5)

for row in ret.collect():
    print ('Dokument: ' + row['text'] +', podobieństwo: '+ str(row['distCol']))

+--------------------+--------------------+
|                text|           tokenized|
+--------------------+--------------------+
|Ala ma psa a nie ...|[ala, ma, psa, a,...|
+--------------------+--------------------+

Dokument: Ala ma kota a kot ma ale, podobieństwo: 0.4285714285714286
Dokument: Hania ma kangura ale nie ma kota, podobieństwo: 0.625


### Zadanie 4 (duże)

Pobierz zbior danych z rosyjskiej gazety pravda [link](http://fizyka.umk.pl/~mich/pravda_extracted.tar.gz) zebranej podczas ataku Rosji na Ukraine. 
* wczytaj te dokumenty do DataFrame
* zaaplikuj tokenizacje (sprobuje RegexTokenizer)
* usun duplikaty uzywajac MinHashLSH